In [1]:
import pandas as pd
from sqlalchemy import create_engine

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from io import StringIO
import numpy as np

### Extract CSVs into DataFrames

In [2]:
# Merging WTI, KWH, Houston Weather

In [3]:
Weather_file = "Resources/Houston_Weather_NOOA_NCDC.csv"
Weather_df = pd.read_csv(Weather_file)
Weather_df.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,...,WT03,WT03_ATTRIBUTES,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT06,WT06_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/1/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/2/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/3/1990,0.03,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/4/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/5/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(Weather_df.columns)

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'PRCP',
       'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'TOBS',
       'TOBS_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT03', 'WT03_ATTRIBUTES',
       'WT04', 'WT04_ATTRIBUTES', 'WT05', 'WT05_ATTRIBUTES', 'WT06',
       'WT06_ATTRIBUTES', 'WT11', 'WT11_ATTRIBUTES'],
      dtype='object')


In [5]:
Weather_df['DATE'] = pd.to_datetime(Weather_df['DATE'])

In [6]:
new_weather_data_df = Weather_df[['DATE','STATION','NAME','TMAX','TMIN','TOBS']]
new_weather_data_df.head()

,DATE,STATION,NAME,TMAX,TMIN,TOBS
0,1990-12-01,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",60.0,44.0,55.0
1,1990-12-02,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",73.0,55.0,60.0
2,1990-12-03,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",76.0,59.0,59.0
3,1990-12-04,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",61.0,36.0,36.0
4,1990-12-05,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",56.0,28.0,30.0


In [7]:
new_weather_data_df.reset_index(drop=True, inplace=True)

In [8]:
new_weather_data_df.to_csv('Resources/Weather_data.csv')

In [9]:
WTI_file = "Resources/WTI_Prices.csv"
WTI_df = pd.read_csv(WTI_file)
WTI_df.head()

,DATE,DCOILWTICO
0,1/14/1986,24.97
1,1/15/1986,25.18
2,1/16/1986,23.98
3,1/17/1986,23.63
4,1/20/1986,21.33


In [10]:
WTI_df['DATE'] = pd.to_datetime(WTI_df['DATE'])
WTI_df.head()

,DATE,DCOILWTICO
0,1986-01-14,24.97
1,1986-01-15,25.18
2,1986-01-16,23.98
3,1986-01-17,23.63
4,1986-01-20,21.33


In [11]:
new_WTI_data = WTI_df[~(WTI_df['DATE'] < '1990-01-01')]
print(new_WTI_data)

           DATE DCOILWTICO
1034 1990-01-01          .
1035 1990-01-02      22.88
1036 1990-01-03      23.81
1037 1990-01-04      23.41
1038 1990-01-05      23.07
...         ...        ...
9255 2021-07-06      73.62
9256 2021-07-07      72.22
9257 2021-07-08      72.98
9258 2021-07-09      74.56
9259 2021-07-12      74.21

[8226 rows x 2 columns]


In [12]:
new_WTI_data_df = pd.DataFrame(new_WTI_data)
new_WTI_data_df

,DATE,DCOILWTICO
1034,1990-01-01,.
1035,1990-01-02,22.88
1036,1990-01-03,23.81
1037,1990-01-04,23.41
1038,1990-01-05,23.07
...,...,...
9255,2021-07-06,73.62
9256,2021-07-07,72.22
9257,2021-07-08,72.98
9258,2021-07-09,74.56


In [13]:
new_WTI_data_df.rename(columns = {'DCOILWTICO':'Price'},inplace = True)

In [14]:
new_WTI_data_df

,DATE,Price
1034,1990-01-01,.
1035,1990-01-02,22.88
1036,1990-01-03,23.81
1037,1990-01-04,23.41
1038,1990-01-05,23.07
...,...,...
9255,2021-07-06,73.62
9256,2021-07-07,72.22
9257,2021-07-08,72.98
9258,2021-07-09,74.56


In [15]:
new_WTI_data_df.reset_index(drop=True, inplace=True)

In [16]:
new_WTI_data_df.drop(new_WTI_data_df[new_WTI_data_df['Price'] == '.'].index, inplace=True)

In [17]:
new_WTI_data_df

,DATE,Price
1,1990-01-02,22.88
2,1990-01-03,23.81
3,1990-01-04,23.41
4,1990-01-05,23.07
5,1990-01-08,21.64
...,...,...
8221,2021-07-06,73.62
8222,2021-07-07,72.22
8223,2021-07-08,72.98
8224,2021-07-09,74.56


In [18]:
new_WTI_data_df.to_csv('Resources/WTI_data.csv')

In [19]:
KWH_file = "Resources/KWH_Houston.csv"
KWH_df = pd.read_csv(KWH_file)
KWH_df.head()

,DATE,APUS37B72610
0,11/1/1978,0.032
1,12/1/1978,0.034
2,1/1/1979,0.035
3,2/1/1979,0.036
4,3/1/1979,0.038


In [20]:
KWH_df.rename(columns = {'APUS37B72610':'Cost_KWH'},inplace = True)

In [21]:
KWH_df['DATE'] = pd.to_datetime(KWH_df['DATE'])
KWH_df.head()

,DATE,Cost_KWH
0,1978-11-01,0.032
1,1978-12-01,0.034
2,1979-01-01,0.035
3,1979-02-01,0.036
4,1979-03-01,0.038


In [22]:
new_KWH_data = KWH_df[~(KWH_df['DATE'] < '1990-01-01')]
print(new_KWH_data)

          DATE Cost_KWH
134 1990-01-01    0.072
135 1990-02-01    0.073
136 1990-03-01    0.073
137 1990-04-01    0.073
138 1990-05-01    0.085
..         ...      ...
507 2021-02-01    0.143
508 2021-03-01    0.144
509 2021-04-01    0.141
510 2021-05-01    0.138
511 2021-06-01    0.138

[378 rows x 2 columns]


In [23]:
new_KWH_df = pd.DataFrame(new_KWH_data) 
new_KWH_df.head()

,DATE,Cost_KWH
134,1990-01-01,0.072
135,1990-02-01,0.073
136,1990-03-01,0.073
137,1990-04-01,0.073
138,1990-05-01,0.085


In [24]:
new_KWH_df.drop(new_KWH_df[new_KWH_df['Cost_KWH'] == '.'].index, inplace=True)

In [25]:
new_KWH_df.applymap(np.isreal)
#new_KWH_df.iloc[0,1]
new_KWH_df['Cost_KWH']=new_KWH_df['Cost_KWH'].astype('float64')

In [26]:
new_KWH_df.to_csv('Resources/KWH_data.csv')

### Transform premise DataFrame

In [35]:
rds_connection_string = "postgres:postgres@localhost:5432/Project_2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [36]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine,reflect=True)
Base

sqlalchemy.ext.automap.Base

In [37]:
engine.table_names()

['kwh', 'wti', 'weather']

In [38]:
from sqlalchemy import inspect

In [39]:
inspector = inspect(engine)
inspector.get_columns('kwh')

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': "nextval('kwh_id_seq'::regclass)",
  'autoincrement': True,
  'comment': None},
 {'name': 'DATE',
  'type': DATE(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Cost_KWH',
  'type': DOUBLE_PRECISION(precision=53),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

### Load DataFrames into database

In [40]:
new_KWH_df.to_sql(name='kwh', con=engine, if_exists='append', index=False)

In [41]:
new_WTI_data_df.to_sql(name='wti', con=engine, if_exists='append', index=False)

In [42]:
new_weather_data_df.to_sql(name='weather', con=engine, if_exists='append', index=False)